In [1]:
!wget http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip

--2024-11-12 13:26:01--  http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip
Resolving weegee.vision.ucmerced.edu (weegee.vision.ucmerced.edu)... 169.236.184.65
Connecting to weegee.vision.ucmerced.edu (weegee.vision.ucmerced.edu)|169.236.184.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 332468434 (317M) [application/zip]
Saving to: ‘UCMerced_LandUse.zip’

UCMerced_LandUse.zi 100%[===================>] 317.07M  30.5MB/s    in 15s     

2024-11-12 13:26:17 (21.1 MB/s) - ‘UCMerced_LandUse.zip’ saved [332468434/332468434]



In [3]:
!unzip /content/UCMerced_LandUse.zip -d dataset

Archive:  /content/UCMerced_LandUse.zip
   creating: dataset/UCMerced_LandUse/Images/
   creating: dataset/UCMerced_LandUse/Images/agricultural/
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural00.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural01.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural02.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural03.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural04.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural05.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural06.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural07.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural08.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricultural09.tif  
  inflating: dataset/UCMerced_LandUse/Images/agricultural/agricul

In [189]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [93]:
def data_load(folder,data_type="train",ration=1):
  catigories={}
  for filename in os.listdir(folder ):
    carigorie=[]
    path =folder + "/" +filename
    if data_type=="train":
      images =os.listdir(path)
    elif data_type=="test":
      images =os.listdir(path)[::-1]
    num_images=int(ration*len(images))

    for idx, image in enumerate(images):
      if idx==num_images:
        break
      img=cv2.imread(path + "/" + image,0)
      if img is not None :
        carigorie.append(img)
    catigories[filename]=carigorie
  return catigories


In [107]:
train_data=data_load("/content/dataset/UCMerced_LandUse/Images","train",0.8)
test_data=data_load("/content/dataset/UCMerced_LandUse/Images","test",0.2)

In [95]:
print(train_data.keys())
print(len(train_data["airplane"]))

dict_keys(['airplane', 'buildings', 'mediumresidential', 'freeway', 'sparseresidential', 'tenniscourt', 'mobilehomepark', 'storagetanks', 'runway', 'golfcourse', 'beach', 'forest', 'harbor', 'baseballdiamond', 'river', 'parkinglot', 'intersection', 'agricultural', 'denseresidential', 'chaparral', 'overpass'])
80


In [96]:
print(test_data.keys())
print(len(test_data["airplane"]))

dict_keys(['airplane', 'buildings', 'mediumresidential', 'freeway', 'sparseresidential', 'tenniscourt', 'mobilehomepark', 'storagetanks', 'runway', 'golfcourse', 'beach', 'forest', 'harbor', 'baseballdiamond', 'river', 'parkinglot', 'intersection', 'agricultural', 'denseresidential', 'chaparral', 'overpass'])
20


In [131]:
def sift_feature(images):
  descriptors_list=[]
  sift_feature={}
  sift =cv2.SIFT_create()
  for key,value in images.items():
    # print(key)
    feature=[]
    i=0
    for image in value :
      try :
        i+=1
        kp,des=sift.detectAndCompute(image,None)
        descriptors_list.extend(des)
        feature.append(des)
      except :
        print(i)
    sift_feature[key]=feature
  return [descriptors_list,sift_feature]







In [132]:
descriptors_list,train_feature=sift_feature(train_data)
_,test_feature=sift_feature(test_data)

65
23
34
6


In [138]:
train_feature

{'airplane': [array([[ 0.,  0.,  0., ..., 13., 19., 11.],
         [ 0.,  0.,  1., ...,  2.,  4., 18.],
         [16.,  1.,  0., ...,  0.,  0.,  0.],
         ...,
         [51., 37.,  1., ...,  1.,  0.,  1.],
         [13., 21.,  3., ...,  0.,  0.,  6.],
         [ 0.,  0.,  0., ...,  1.,  3., 33.]], dtype=float32),
  array([[ 1., 20., 46., ...,  4.,  5.,  5.],
         [11.,  1.,  3., ...,  2.,  4.,  6.],
         [17.,  2.,  1., ...,  2.,  3.,  2.],
         ...,
         [34., 27.,  3., ...,  2.,  1., 25.],
         [21.,  1.,  1., ...,  3.,  2., 11.],
         [ 2.,  9., 15., ..., 19.,  7., 18.]], dtype=float32),
  array([[  3., 137., 128., ...,   0.,   0.,   0.],
         [  0.,   0.,   1., ...,   0.,   6.,   1.],
         [  4.,   0.,   0., ...,   0.,   0.,   4.],
         ...,
         [  3.,   8.,   2., ...,   3.,   1.,   1.],
         [  3.,   3.,   1., ...,   7.,   2.,  14.],
         [  5.,   7.,  57., ...,   0.,   0.,   7.]], dtype=float32),
  array([[ 99.,   4.,   0., ...

In [139]:
def clustirng(k,descriptors_list):
  Kmeans=KMeans(100, init='k-means++')
  Kmeans.fit(descriptors_list)
  visual_word=Kmeans.cluster_centers_
  return visual_word

In [140]:
bow=clustirng(100,descriptors_list)

In [141]:
len(bow)

100

In [142]:
def image_classes(all_des,center):
  catigories={}
  for key ,value in all_des.items():
    catigorie=[]
    for image in value :
      histogram=np.zeros(len(center))
      for fearure in image :
        diffrent =np.absolute(center-fearure)
        magnitude=np.linalg.norm(diffrent,axis=-1)
        index=magnitude.argmin()
        histogram[index]+=1
      catigorie.append(histogram)
    catigories[key]=catigorie
  return catigories



In [143]:
train_bovw=image_classes(train_feature,bow)
test_bovw=image_classes(test_feature,bow)

In [145]:
train_bovw

{'airplane': [array([ 1.,  1.,  2.,  3.,  6.,  0.,  2.,  1.,  2.,  0.,  2.,  6.,  0.,
          2.,  2.,  2.,  0.,  2.,  3.,  3.,  4.,  1.,  0.,  2.,  4., 25.,
          3.,  4.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  2.,  0.,  2.,  0.,
          0.,  1.,  3.,  3.,  9., 11.,  3.,  0.,  0.,  7.,  3.,  1.,  2.,
          2.,  1.,  1.,  1.,  1.,  1.,  1.,  4.,  2.,  1.,  2.,  1.,  2.,
          3.,  0.,  0.,  3., 21.,  2.,  0.,  3.,  1.,  2.,  1.,  1.,  0.,
          1.,  0.,  1.,  0.,  8.,  5.,  0.,  1., 11.,  1.,  0.,  5.,  4.,
          1.,  2.,  2.,  1., 11.,  2.,  1.,  0.,  1.]),
  array([ 2.,  4.,  1., 26.,  5.,  0.,  1.,  2.,  1.,  0.,  0.,  2.,  0.,
          0.,  0.,  2.,  2.,  1.,  8.,  1.,  0.,  0.,  0.,  2.,  1., 11.,
          0.,  1.,  0.,  0.,  4.,  2.,  2.,  0.,  0.,  1.,  1.,  1.,  0.,
          0.,  0.,  2.,  1.,  4.,  3.,  2.,  1.,  1.,  2.,  3.,  1.,  3.,
          2.,  0.,  1.,  1.,  2.,  0.,  3.,  3.,  2.,  3.,  1.,  0.,  0.,
          1.,  1.,  0.,  0., 11.,  2.,  1., 

In [195]:
flatten_data=[(cls,*feature) for cls ,features in train_bovw.items() for feature in features ]
train_dataframe=pd.DataFrame(flatten_data)
train_dataframe=train_dataframe.sample(len(train_dataframe))
train_dataframe

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,airplane,1.0,1.0,2.0,3.0,6.0,0.0,2.0,1.0,2.0,...,4.0,1.0,2.0,2.0,1.0,11.0,2.0,1.0,0.0,1.0
1,airplane,2.0,4.0,1.0,26.0,5.0,0.0,1.0,2.0,1.0,...,4.0,0.0,2.0,0.0,2.0,3.0,1.0,1.0,1.0,1.0
2,airplane,6.0,6.0,3.0,3.0,19.0,0.0,4.0,8.0,11.0,...,6.0,5.0,3.0,8.0,9.0,3.0,10.0,14.0,4.0,1.0
3,airplane,0.0,1.0,0.0,11.0,6.0,0.0,0.0,3.0,0.0,...,2.0,0.0,3.0,2.0,0.0,2.0,0.0,4.0,0.0,0.0
4,airplane,1.0,1.0,4.0,6.0,0.0,0.0,3.0,2.0,0.0,...,0.0,1.0,0.0,1.0,2.0,4.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,overpass,0.0,1.0,4.0,1.0,0.0,10.0,1.0,1.0,0.0,...,2.0,2.0,0.0,1.0,0.0,3.0,1.0,4.0,0.0,0.0
1673,overpass,5.0,1.0,1.0,7.0,2.0,1.0,3.0,2.0,1.0,...,3.0,3.0,3.0,2.0,1.0,3.0,3.0,1.0,1.0,3.0
1674,overpass,3.0,0.0,1.0,6.0,0.0,2.0,0.0,2.0,1.0,...,0.0,0.0,5.0,4.0,1.0,1.0,6.0,1.0,6.0,2.0
1675,overpass,2.0,2.0,0.0,0.0,2.0,7.0,1.0,0.0,0.0,...,3.0,1.0,1.0,0.0,0.0,2.0,0.0,4.0,0.0,4.0


In [196]:
flatten_data_test=[(cls,*feature) for cls ,features in test_bovw.items() for feature in features ]
test_dataframe=pd.DataFrame(flatten_data_test)
test_dataframe=test_dataframe.sample(len(test_dataframe))
test_dataframe

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
80,sparseresidential,10.0,15.0,5.0,9.0,13.0,0.0,25.0,18.0,16.0,...,19.0,12.0,9.0,16.0,12.0,6.0,19.0,10.0,15.0,15.0
171,runway,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,1.0,0.0
258,harbor,4.0,12.0,25.0,1.0,2.0,3.0,36.0,14.0,10.0,...,13.0,13.0,5.0,9.0,18.0,14.0,3.0,8.0,8.0,19.0
296,river,20.0,12.0,11.0,0.0,12.0,0.0,8.0,10.0,11.0,...,7.0,11.0,3.0,10.0,9.0,4.0,11.0,8.0,12.0,8.0
260,baseballdiamond,1.0,4.0,0.0,8.0,0.0,0.0,3.0,0.0,0.0,...,3.0,1.0,2.0,0.0,4.0,5.0,2.0,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,freeway,1.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
352,agricultural,0.0,4.0,6.0,0.0,0.0,257.0,10.0,0.0,1.0,...,4.0,7.0,0.0,1.0,16.0,4.0,0.0,0.0,10.0,4.0
350,agricultural,0.0,0.0,0.0,0.0,0.0,585.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
368,denseresidential,9.0,10.0,10.0,9.0,8.0,0.0,3.0,6.0,10.0,...,15.0,4.0,9.0,4.0,5.0,11.0,4.0,12.0,4.0,2.0


In [198]:
x_train=train_dataframe.drop(columns=[0])
y_train=train_dataframe[0]
x_test=test_dataframe.drop(columns=[0])
y_test=test_dataframe[0]

In [199]:
le=LabelEncoder()
transformed_y_train=le.fit_transform(y_train)
transformed_y_test=le.transform(y_test)

In [200]:
linear_svc = SVC(kernel='linear')
linear_svc.fit(x_train,transformed_y_train)
train_prediction=linear_svc.predict(x_train)
train_accuracy=accuracy_score(train_prediction,transformed_y_train)
print("accuracy on training dataset is : " ,train_accuracy)

accuracy on training dataset is :  1.0


In [201]:
test_prediction=linear_svc.predict(x_test)
test_accuracy=accuracy_score(test_prediction,transformed_y_test)
print("accuracy on testing dataset is : " ,test_accuracy)

accuracy on testing dataset is :  0.6539379474940334
